In [2]:
import pandas as pd

In [3]:
maestra_path = "C:/Users/Rodrigo/Desktop/Roxana Files/Add databases/Github ANID/maestra.xlsx"
anid_path = "C:/Users/Rodrigo/Desktop/Roxana Files/Add databases/Github ANID/BDH_HISTORICA.xlsx"
output_path_csv = "C:/Users/Rodrigo/Desktop/Roxana Files/Add databases/Github ANID/maestra_anidgithub.csv"
output_path_xlsx = "C:/Users/Rodrigo/Desktop/Roxana Files/Add databases/Github ANID/maestra_anidgithub.xlsx"

In [4]:
def cargar_y_filtrar_anid(anid_path):
    """
    Carga y filtra la base de datos ANID según los criterios indicados para FONDECYT.

    Args:
    anid_path (str): Ruta del archivo Excel de la base de datos ANID.

    Returns:
    pd.DataFrame: DataFrame filtrado de la base ANID.
    """
    # Cargar base de datos ANID
    anid = pd.read_excel(anid_path)
    
    # Filtrar por criterios FONDECYT e INSTRUMENTO
    anid_filtrada = anid[(anid['PROGRAMA'] == 'FONDECYT') &
                         (anid['INSTRUMENTO'].isin(['REGULAR', 'POSTDOCTORADO', 'INICIACION'])) &
                         (anid['AGNO_FALLO'].between(2014, 2024))]

    # Normalizar nombres en ANID
    anid_filtrada['NOMBRE_RESPONSABLE'] = anid_filtrada['NOMBRE_RESPONSABLE'].str.upper().str.strip()
    
    return anid_filtrada

In [5]:
def normalizar_y_crear_combinaciones(maestra_path):
    """
    Normaliza los nombres en la base de datos maestra y crea combinaciones de nombres.

    Args:
    maestra_path (str): Ruta del archivo Excel de la base de datos maestra.

    Returns:
    pd.DataFrame: DataFrame con combinaciones de nombres generadas.
    """
    # Cargar la base de datos maestra
    maestra = pd.read_excel(maestra_path, sheet_name='BaseMaestra')
    
    # Normalizar nombres a mayúsculas y limpiar espacios en la base maestra
    columnas_nombres = ['Primer_Nombre', 'Segundo_Nombre', 'Apellido_Paterno', 'Apellido_Materno']
    maestra[columnas_nombres] = maestra[columnas_nombres].apply(lambda x: x.str.upper().str.strip())
    
    # Crear combinaciones de nombres
    maestra = maestra.assign(
        comb_1=maestra['Primer_Nombre'] + " " + maestra['Segundo_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
        comb_2=maestra['Primer_Nombre'] + " " + maestra['Segundo_Nombre'] + " " + maestra['Apellido_Materno'] + " " + maestra['Apellido_Paterno'],
        comb_3=maestra['Primer_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
        comb_4=maestra['Primer_Nombre'] + " " + maestra['Apellido_Materno'] + " " + maestra['Apellido_Paterno'],
        comb_5=maestra['Segundo_Nombre'] + " " + maestra['Primer_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
        comb_6=maestra['Segundo_Nombre'] + " " + maestra['Primer_Nombre'] + " " + maestra['Apellido_Materno'] + " " + maestra['Apellido_Paterno'],
        comb_7=maestra['Segundo_Nombre'] + " " + maestra['Apellido_Paterno'] + " " + maestra['Apellido_Materno'],
        comb_8=maestra['Segundo_Nombre'] + " " + maestra['Apellido_Materno'] + " " + maestra['Apellido_Paterno'],
        comb_9=maestra['Primer_Nombre'] + " " + maestra['Segundo_Nombre'] + " " + maestra['Apellido_Materno'],
        comb_10=maestra['Segundo_Nombre'] + " " + maestra['Primer_Nombre'] + " " + maestra['Apellido_Materno'],
        comb_11=maestra['Primer_Nombre'] + " " + maestra['Segundo_Nombre'] + " " + maestra['Apellido_Paterno'],
        comb_12=maestra['Segundo_Nombre'] + " " + maestra['Primer_Nombre'] + " " + maestra['Apellido_Paterno']
    )
    
    return maestra

In [6]:
def hacer_joins_y_logs(maestra, anid_filtrada):
    """
    Realiza el join para cada combinación de nombres y guarda los resultados con logs.

    Args:
    maestra (pd.DataFrame): DataFrame de la base de datos maestra con combinaciones de nombres.
    anid_filtrada (pd.DataFrame): DataFrame filtrado de la base ANID.

    Returns:
    pd.DataFrame: DataFrame con los resultados combinados y eliminando duplicados.
    """
    resultados = []
    total_coincidencias = 0
    
    # Iterar sobre las combinaciones
    for i in range(1, 13):
        comb_name = f'comb_{i}'
        # Realizar el join con la combinación actual
        resultado = pd.merge(maestra, anid_filtrada, how='inner', left_on=comb_name, right_on='NOMBRE_RESPONSABLE')
        resultados.append(resultado)
        
        # Log de coincidencias
        coincidencias = len(resultado)
        total_coincidencias += coincidencias
        print(f"Coincidencias en combinación {comb_name}: {coincidencias}")
    
    # Unir todos los resultados en un solo DataFrame
    resultados_unidos = pd.concat(resultados, ignore_index=True)
    resultados_unidos = resultados_unidos.drop_duplicates()

    print(f"Total de coincidencias en todas las combinaciones: {total_coincidencias}")
    
    return resultados_unidos

In [7]:
def guardar_resultados(resultados_unidos, output_path_csv, output_path_xlsx):
    """
    Guarda la base de datos resultante en formato CSV y Excel.

    Args:
    resultados_unidos (pd.DataFrame): DataFrame con los resultados de todas las combinaciones.
    output_path_csv (str): Ruta de salida para guardar el archivo CSV.
    output_path_xlsx (str): Ruta de salida para guardar el archivo Excel.
    
    Returns:
    None
    """
    # Exportar los resultados a un archivo CSV
    resultados_unidos.to_csv(output_path_csv, index=False)
    print(f"Base de datos guardada en formato CSV en: {output_path_csv}")
    
    # Exportar los resultados a un archivo Excel
    resultados_unidos.to_excel(output_path_xlsx, index=False)
    print(f"Base de datos guardada en formato Excel en: {output_path_xlsx}")


In [8]:
anid_filtrada = cargar_y_filtrar_anid(anid_path)

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_5708\308634130.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anid_filtrada['NOMBRE_RESPONSABLE'] = anid_filtrada['NOMBRE_RESPONSABLE'].str.upper().str.strip()


In [9]:
anid_filtrada.head(5)

,N,CODIGO_PROYECTO,SUBDIRECCION,PROGRAMA,INSTRUMENTO,NOMBRE_CONCURSO,AGNO_CONCURSO,AGNO_FALLO,NOMBRE_PROYECTO,AREA_OCDE,...,TIPO_BENEFICIARIO,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL,MACROZONA_MINCIENCIA,REGION_EJECUCION,MONTO_ADJUDICADO,NOTA_MONTO,MONEDA,PALABRAS_CLAVES
23884,23885,3140001,PROYECTOS DE INVESTIGACION,FONDECYT,POSTDOCTORADO,CONCURSO POSTDOCTORADO 2014,2013.0,2014,ENCAPSULAMIENTO DE AROMAS: EVALUACION DE MICRO...,INGENIERIA Y TECNOLOGIA,...,PERSONA NATURAL,CESAR HORACIO BURGOS DIAZ,HOMBRE,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...,SUR,REGIÓN DE LA ARAUCANÍA,71352,NaN,Miles de pesos (M$),SIN INFORMACION
23885,23886,3140002,PROYECTOS DE INVESTIGACION,FONDECYT,POSTDOCTORADO,CONCURSO POSTDOCTORADO 2014,2013.0,2014,ESTUDIO TEORICO DE INTERACCION DE COMPLEJOS HE...,CIENCIAS NATURALES,...,PERSONA NATURAL,LEONOR ANGELINA ALVARADO SOTO,MUJER,UNIVERSIDAD BERNARDO OHIGGINS,RM,REGIÓN METROPOLITANA DE SANTIAGO,71352,NaN,Miles de pesos (M$),SIN INFORMACION
23886,23887,3140003,PROYECTOS DE INVESTIGACION,FONDECYT,POSTDOCTORADO,CONCURSO POSTDOCTORADO 2014,2013.0,2014,EVALUACION DE LA PARTICIPACION DE COMPONENTES ...,CIENCIAS NATURALES,...,PERSONA NATURAL,MATIAS ESTEBAN CASTRO GONZALEZ,HOMBRE,FUNDACION CIENCIA PARA LA VIDA,RM,REGIÓN METROPOLITANA DE SANTIAGO,71352,NaN,Miles de pesos (M$),SIN INFORMACION
23887,23888,3140005,PROYECTOS DE INVESTIGACION,FONDECYT,POSTDOCTORADO,CONCURSO POSTDOCTORADO 2014,2013.0,2014,DEVELOPMENT OF AN INTEGRATIVE PIPELINE FOR MOB...,CIENCIAS NATURALES,...,PERSONA NATURAL,FRANCISCO JAVIER OSSANDON CABRERA,HOMBRE,FUNDACION CIENCIA PARA LA VIDA,RM,REGIÓN METROPOLITANA DE SANTIAGO,46692,NaN,Miles de pesos (M$),SIN INFORMACION
23888,23889,3140006,PROYECTOS DE INVESTIGACION,FONDECYT,POSTDOCTORADO,CONCURSO POSTDOCTORADO 2014,2013.0,2014,INTERFERENCIA DE NCMTRNAS DE ORIGEN CANINO COM...,CIENCIAS NATURALES,...,PERSONA NATURAL,NICOLAS PABLO CIFUENTES MUÑOZ,HOMBRE,FUNDACION CIENCIA PARA LA VIDA,RM,REGIÓN METROPOLITANA DE SANTIAGO,70248,NaN,Miles de pesos (M$),SIN INFORMACION


In [10]:
maestra = normalizar_y_crear_combinaciones(maestra_path)

maestra

,FOLIO,NOMBRE,Primer_Nombre,Segundo_Nombre,Apellido_Paterno,Apellido_Materno,SEXO_COMPLETO,ESTADO BECA,Ano de inicio beca,TIPO_BECA,...,comb_3,comb_4,comb_5,comb_6,comb_7,comb_8,comb_9,comb_10,comb_11,comb_12
0,72160230,ABELINO ENRIQUE JIMENEZ GAJARDO,ABELINO,ENRIQUE,JIMENEZ,GAJARDO,MASCULINO,NO VIGENTE,2015,BECAS CHILE,...,ABELINO JIMENEZ GAJARDO,ABELINO GAJARDO JIMENEZ,ENRIQUE ABELINO JIMENEZ GAJARDO,ENRIQUE ABELINO GAJARDO JIMENEZ,ENRIQUE JIMENEZ GAJARDO,ENRIQUE GAJARDO JIMENEZ,ABELINO ENRIQUE GAJARDO,ENRIQUE ABELINO GAJARDO,ABELINO ENRIQUE JIMENEZ,ENRIQUE ABELINO JIMENEZ
1,21150571,ABRAHAM BENJAMIN NOVOA LAGOS,ABRAHAM,BENJAMIN,NOVOA,LAGOS,MASCULINO,NO VIGENTE,2015,CONICYT,...,ABRAHAM NOVOA LAGOS,ABRAHAM LAGOS NOVOA,BENJAMIN ABRAHAM NOVOA LAGOS,BENJAMIN ABRAHAM LAGOS NOVOA,BENJAMIN NOVOA LAGOS,BENJAMIN LAGOS NOVOA,ABRAHAM BENJAMIN LAGOS,BENJAMIN ABRAHAM LAGOS,ABRAHAM BENJAMIN NOVOA,BENJAMIN ABRAHAM NOVOA
2,72170568,ACCEL NICOLAS ABARCA PROUZA,ACCEL,NICOLAS,ABARCA,PROUZA,MASCULINO,NO VIGENTE,2016,BECAS CHILE,...,ACCEL ABARCA PROUZA,ACCEL PROUZA ABARCA,NICOLAS ACCEL ABARCA PROUZA,NICOLAS ACCEL PROUZA ABARCA,NICOLAS ABARCA PROUZA,NICOLAS PROUZA ABARCA,ACCEL NICOLAS PROUZA,NICOLAS ACCEL PROUZA,ACCEL NICOLAS ABARCA,NICOLAS ACCEL ABARCA
3,72170089,ADELAIDA BARROS CAJDLER,ADELAIDA,BARROS,BARROS,NaN,FEMENINO,NO VIGENTE,2016,BECAS CHILE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADELAIDA BARROS BARROS,BARROS ADELAIDA BARROS
4,72170561,ADOLFO ANTONIO ROCCO ROCCO,ADOLFO,ANTONIO,ROCCO,ROCCO,MASCULINO,NO VIGENTE,2016,BECAS CHILE,...,ADOLFO ROCCO ROCCO,ADOLFO ROCCO ROCCO,ANTONIO ADOLFO ROCCO ROCCO,ANTONIO ADOLFO ROCCO ROCCO,ANTONIO ROCCO ROCCO,ANTONIO ROCCO ROCCO,ADOLFO ANTONIO ROCCO,ANTONIO ADOLFO ROCCO,ADOLFO ANTONIO ROCCO,ANTONIO ADOLFO ROCCO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3161,21161551,ZACHARY JOHN HUGO,ZACHARY,JOHN,JOHN,NaN,MASCULINO,NO VIGENTE,2016,CONICYT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZACHARY JOHN JOHN,JOHN ZACHARY JOHN
3162,21151192,ZAMBRA DEL CARMEN LOPEZ FARRAN,ZAMBRA,DEL,LOPEZ,NaN,FEMENINO,NO VIGENTE,2015,CONICYT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZAMBRA DEL LOPEZ,DEL ZAMBRA LOPEZ
3163,72150054,ZAYRA ELIZABETH ANTUNEZ SANHUEZA,ZAYRA,ELIZABETH,ANTUNEZ,SANHUEZA,FEMENINO,NO VIGENTE,2014,BECAS CHILE,...,ZAYRA ANTUNEZ SANHUEZA,ZAYRA SANHUEZA ANTUNEZ,ELIZABETH ZAYRA ANTUNEZ SANHUEZA,ELIZABETH ZAYRA SANHUEZA ANTUNEZ,ELIZABETH ANTUNEZ SANHUEZA,ELIZABETH SANHUEZA ANTUNEZ,ZAYRA ELIZABETH SANHUEZA,ELIZABETH ZAYRA SANHUEZA,ZAYRA ELIZABETH ANTUNEZ,ELIZABETH ZAYRA ANTUNEZ
3164,63140267,ZENEIDA ELIZABETH WONG CHANG,ZENEIDA,ELIZABETH,WONG,CHANG,FEMENINO,NO VIGENTE,2014,CONICYT,...,ZENEIDA WONG CHANG,ZENEIDA CHANG WONG,ELIZABETH ZENEIDA WONG CHANG,ELIZABETH ZENEIDA CHANG WONG,ELIZABETH WONG CHANG,ELIZABETH CHANG WONG,ZENEIDA ELIZABETH CHANG,ELIZABETH ZENEIDA CHANG,ZENEIDA ELIZABETH WONG,ELIZABETH ZENEIDA WONG


In [11]:
resultados_unidos = hacer_joins_y_logs(maestra, anid_filtrada)

resultados_unidos

Coincidencias en combinación comb_1: 200
Coincidencias en combinación comb_2: 2
Coincidencias en combinación comb_3: 486
Coincidencias en combinación comb_4: 6
Coincidencias en combinación comb_5: 28
Coincidencias en combinación comb_6: 0
Coincidencias en combinación comb_7: 7
Coincidencias en combinación comb_8: 9
Coincidencias en combinación comb_9: 0
Coincidencias en combinación comb_10: 0
Coincidencias en combinación comb_11: 2
Coincidencias en combinación comb_12: 0
Total de coincidencias en todas las combinaciones: 740


,FOLIO,NOMBRE,Primer_Nombre,Segundo_Nombre,Apellido_Paterno,Apellido_Materno,SEXO_COMPLETO,ESTADO BECA,Ano de inicio beca,TIPO_BECA,...,TIPO_BENEFICIARIO_y,NOMBRE_RESPONSABLE,SEXO,INSTITUCION_PRINCIPAL_y,MACROZONA_MINCIENCIA_y,REGION_EJECUCION_y,MONTO_ADJUDICADO_y,NOTA_MONTO_y,MONEDA_y,PALABRAS_CLAVES_y
0,21140234,ALEJANDRA ESTELA VELASQUEZ ORDENES,ALEJANDRA,ESTELA,VELASQUEZ,ORDENES,FEMENINO,NO VIGENTE,2014,CONICYT,...,PERSONA NATURAL,ALEJANDRA ESTELA VELASQUEZ ORDENES,MUJER,UNIVERSIDAD DE CONCEPCION,CENTRO SUR,REGIÓN DEL BIOBÍO,78624,NaN,Miles de pesos (M$),ENDOMETRIO BOVINO;MATRIZ TRIDIMENSIONAL (3D);D...
1,72150221,ALEJANDRO ANDRES ALBORNOZ ROJAS,ALEJANDRO,ANDRES,ALBORNOZ,ROJAS,MASCULINO,NO VIGENTE,2014,BECAS CHILE,...,PERSONA NATURAL,ALEJANDRO ANDRES ALBORNOZ ROJAS,HOMBRE,UNIVERSIDAD AUSTRAL DE CHILE,SUR,REGIÓN DE LOS RÍOS,93168,NaN,Miles de pesos (M$),SPATIAL COMPOSITION ANALYSIS; ACOUSMATIC MUSIC...
2,72160165,ALEJANDRO GABRIEL IANISZEWSKI GOMEZ,ALEJANDRO,GABRIEL,IANISZEWSKI,GOMEZ,MASCULINO,NO VIGENTE,2015,BECAS CHILE,...,PERSONA NATURAL,ALEJANDRO GABRIEL IANISZEWSKI GOMEZ,HOMBRE,UNIVERSIDAD SAN SEBASTIÁN,RM,REGIÓN METROPOLITANA DE SANTIAGO,104194,NaN,Miles de pesos (M$),SPEECH PERCEPTION IN NOISE; PERIPHERAL AND SUB...
3,21151024,ALEJANDRO JOSE VALENZUELA ALDRIDGE,ALEJANDRO,JOSE,VALENZUELA,ALDRIDGE,MASCULINO,NO VIGENTE,2015,CONICYT,...,PERSONA NATURAL,ALEJANDRO JOSE VALENZUELA ALDRIDGE,HOMBRE,UNIVERSIDAD ADOLFO IBÁÑEZ,RM,REGIÓN METROPOLITANA DE SANTIAGO,40926,NaN,Miles de pesos (M$),AUTONOMÍA LITERARIA; JOSÉ VASCONCELOS; JOSÉ CA...
4,21160371,ALFREDO ANTONIO ALEGRIA JIMENEZ,ALFREDO,ANTONIO,ALEGRIA,JIMENEZ,MASCULINO,NO VIGENTE,2016,CONICYT,...,PERSONA NATURAL,ALFREDO ANTONIO ALEGRIA JIMENEZ,HOMBRE,UNIVERSIDAD TECNICA FEDERICO SANTA MARIA,CENTRO,REGIÓN DE VALPARAÍSO,51120,NaN,Miles de pesos (M$),CROSS-COVARIANCE FUNCTIONS;POSITIVE DEFINITE F...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,72160368,RODRIGO ARIEL LOPEZ ORELLANA,RODRIGO,ARIEL,LOPEZ,ORELLANA,MASCULINO,NO VIGENTE,2015,BECAS CHILE,...,PERSONA NATURAL,ARIEL ORELLANA LOPEZ,HOMBRE,UNIVERSIDAD NACIONAL ANDRES BELLO,RM,REGIÓN METROPOLITANA DE SANTIAGO,270000,NaN,Miles de pesos (M$),NUCLEOTIDE SUGAR TRANSPORTERS;PLANT CELL WALL;...
736,72160368,RODRIGO ARIEL LOPEZ ORELLANA,RODRIGO,ARIEL,LOPEZ,ORELLANA,MASCULINO,NO VIGENTE,2015,BECAS CHILE,...,PERSONA NATURAL,ARIEL ORELLANA LOPEZ,HOMBRE,UNIVERSIDAD ANDRES BELLO,RM,REGIÓN METROPOLITANA DE SANTIAGO,267388,NaN,Miles de pesos (M$),PLANT CELL WALL;SAM TRANSPORT;POLYSACCHARIDE M...
737,72160368,RODRIGO ARIEL LOPEZ ORELLANA,RODRIGO,ARIEL,LOPEZ,ORELLANA,MASCULINO,NO VIGENTE,2015,BECAS CHILE,...,PERSONA NATURAL,ARIEL ORELLANA LOPEZ,HOMBRE,UNIVERSIDAD NACIONAL ANDRES BELLO,RM,REGIÓN METROPOLITANA DE SANTIAGO,270000,NaN,Miles de pesos (M$),NUCLEOTIDE SUGAR TRANSPORTERS;PLANT CELL WALL;...
738,72150412,IRENE DEL REAL CONTRERAS,IRENE,DEL,REAL,CONTRERAS,FEMENINO,NO VIGENTE,2014,BECAS CHILE,...,PERSONA NATURAL,IRENE DEL REAL,MUJER,UNIVERSIDAD DE CHILE,RM,REGIÓN METROPOLITANA DE SANTIAGO,80134,NaN,Miles de pesos (M$),IOCG EXPLORATION PATHFINDERS;ACTINOLITE ELEMEN...


In [12]:
guardar_resultados(resultados_unidos, output_path_csv, output_path_xlsx)

Base de datos guardada en formato CSV en: C:/Users/Rodrigo/Desktop/Roxana Files/Add databases/Github ANID/maestra_anidgithub.csv
Base de datos guardada en formato Excel en: C:/Users/Rodrigo/Desktop/Roxana Files/Add databases/Github ANID/maestra_anidgithub.xlsx
